In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from googletrans import Translator
import time
import numpy as np

In [103]:
#Parámetros de búsqueda y extracción

url_basica="https://www.whocc.no/atc_ddd_index/?code="
fin="&showdescription=no"
letras1=["A","B","C","D","G","H","J","L","M","N","P","R","S","V"]
letras2=[chr(i) for i in range(65,91)]
letras3=[chr(i) for i in range(65,91)]
#Números en string del "01" al "99"
numeros=[str(i).zfill(2) for i in range(1,100)]
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

#Se crean urls
urls=[]
for letra1 in letras1:
    for number in numeros:
        for letra2 in letras2:
            for letra3 in letras3:
                urls.append(url_basica+letra1+number+letra2+letra3+fin)
print("El número de urls potenciales es: ",len(urls))


El número de urls potenciales es:  936936


In [86]:
#Obtener la información de todas las urls

all_category_codes = []
all_category_names = []
all_data = []

for url in urls:
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

    # categorías y códigos
    categories = soup.find_all('b')
    category_codes = []
    category_names = []

    for category in categories:
        category_link = category.find('a', href=True)
        if category_link:
            category_code = category_link['href'].split('=')[1].split('&')[0]
            category_name = category_link.text.strip()
            category_codes.append(category_code)
            category_names.append(category_name)
    
    all_category_codes.append(category_codes)
    all_category_names.append(category_names)

    # extraer información de la tabla
    table = soup.find('table')
    try:
        rows = table.find_all('tr')
    except:
        rows = []

    # Pegar todas las filas en una lista
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)
    data = data[1:]

    all_data.append(data)
    time.sleep(2)


In [89]:
dfs = [] # Lista vacía para guardar los dataframes

# iterar sobre las listas de datos, códigos de categoría y nombres de categoría
for data, category_codes, category_names in zip(all_data, all_category_codes, all_category_names):

    # Crear un dataframe
    df = pd.DataFrame(data, columns=['atc5_code', 'drug_name', 'ddd', 'unit', 'administration', 'note'])

    # Crear columnas para el código de categoría y el nombre de categoría
    df['category_code'] = category_codes[0] if category_codes else np.nan
    df['category_name'] = category_names[0] if category_names else np.nan

    # Reemplazar los valores vacíos con NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)

    # Relenar los valores NaN con el valor de la fila anterior
    df['atc5_code'] = df['atc5_code'].fillna(method='ffill')
    df['drug_name'] = df['drug_name'].fillna(method='ffill')

    # Incrementar el código de categoría y el nombre de categoría
    df['atc1_code'] = df['atc5_code'].apply(lambda x: x[0])
    df["atc1_name"] = df["atc1_code"].map(dict(zip(category_codes, category_names)))
    df["atc2_code"] = df["atc5_code"].apply(lambda x: x[:3])
    df["atc2_name"] = df["atc2_code"].map(dict(zip(category_codes, category_names)))
    df["atc3_code"] = df["atc5_code"].apply(lambda x: x[:4])
    df["atc3_name"] = df["atc3_code"].map(dict(zip(category_codes, category_names)))
    df["atc4_code"] = df["atc5_code"].apply(lambda x: x[:5])
    df["atc4_name"] = df["atc4_code"].map(dict(zip(category_codes, category_names)))

    # Agregar el dataframe a la lista de dataframes
    dfs.append(df)

# Concatenar todos los dataframes
df = pd.concat(dfs, ignore_index=True)

# Ordenar las columnas
df = df[['atc1_code', 'atc1_name', 'atc2_code', 'atc2_name', 'atc3_code', 'atc3_name', 'atc4_code', 'atc4_name', 'atc5_code', 'drug_name', 'ddd', 'unit', 'administration', 'note']]

# Minúsculas
df = df.apply(lambda x: x.str.lower() if x.dtype == "object" else x)

df

,atc1_code,atc1_name,atc2_code,atc2_name,atc3_code,atc3_name,atc4_code,atc4_name,atc5_code,drug_name,ddd,unit,administration,note
0,a,alimentary tract and metabolism,a01,stomatological preparations,a01a,stomatological preparations,a01aa,caries prophylactic agents,a01aa01,sodium fluoride,1.1,mg,o,0.5 mg fluoride
1,a,alimentary tract and metabolism,a01,stomatological preparations,a01a,stomatological preparations,a01aa,caries prophylactic agents,a01aa02,sodium monofluorophosphate,NaN,NaN,NaN,NaN
2,a,alimentary tract and metabolism,a01,stomatological preparations,a01a,stomatological preparations,a01aa,caries prophylactic agents,a01aa03,olaflur,1.1,mg,o,NaN
3,a,alimentary tract and metabolism,a01,stomatological preparations,a01a,stomatological preparations,a01aa,caries prophylactic agents,a01aa04,stannous fluoride,NaN,NaN,NaN,NaN
4,a,alimentary tract and metabolism,a01,stomatological preparations,a01a,stomatological preparations,a01aa,caries prophylactic agents,a01aa30,combinations,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,b,blood and blood forming organs,b02,antihemorrhagics,b02b,vitamin k and other hemostatics,b02ba,vitamin k,b02ba01,phytomenadione,20,mg,o,NaN
93,b,blood and blood forming organs,b02,antihemorrhagics,b02b,vitamin k and other hemostatics,b02ba,vitamin k,b02ba01,phytomenadione,20,mg,p,NaN
94,b,blood and blood forming organs,b02,antihemorrhagics,b02b,vitamin k and other hemostatics,b02ba,vitamin k,b02ba02,menadione,10,mg,o,NaN
95,b,blood and blood forming organs,b02,antihemorrhagics,b02b,vitamin k and other hemostatics,b02ba,vitamin k,b02ba02,menadione,2,mg,p,NaN


In [92]:
#Traducir a español toda la tabla con googletrans

translator = Translator()
#Crear copia del dataframe
df_esp=df.copy()
df_esp["drug_name"]=df_esp["drug_name"].apply(lambda x: translator.translate(x,dest="es").text)
df_esp["atc1_name"]=df_esp["atc1_name"].apply(lambda x: translator.translate(x,dest="es").text)
df_esp["atc2_name"]=df_esp["atc2_name"].apply(lambda x: translator.translate(x,dest="es").text)
df_esp["atc3_name"]=df_esp["atc3_name"].apply(lambda x: translator.translate(x,dest="es").text)
df_esp["atc4_name"]=df_esp["atc4_name"].apply(lambda x: translator.translate(x,dest="es").text)
df_esp["atc5_code"]=df_esp["atc5_code"].apply(lambda x: translator.translate(x,dest="es").text)
df_esp["administration"]=df_esp["administration"].apply(lambda x: translator.translate(x,dest="es").text)
df_esp["note"]=df_esp["note"].apply(lambda x: translator.translate(x,dest="es").text)
df_esp

,atc1_code,atc1_name,atc2_code,atc2_name,atc3_code,atc3_name,atc4_code,atc4_name,atc5_code,drug_name,ddd,unit,administration,note
0,a,tracto alimentario y metabolismo,a01,preparaciones estomatológicas,a01a,preparaciones estomatológicas,a01aa,agentes profilácticos de caries,a01aa01,fluoruro de sodio,1.1,mg,O,"0,5 mg de fluoruro"
1,a,tracto alimentario y metabolismo,a01,preparaciones estomatológicas,a01a,preparaciones estomatológicas,a01aa,agentes profilácticos de caries,a01aa02,monofluorofosfato de sodio,NaN,NaN,en,en
2,a,tracto alimentario y metabolismo,a01,preparaciones estomatológicas,a01a,preparaciones estomatológicas,a01aa,agentes profilácticos de caries,a01aa03,olafluro,1.1,mg,O,en
3,a,tracto alimentario y metabolismo,a01,preparaciones estomatológicas,a01a,preparaciones estomatológicas,a01aa,agentes profilácticos de caries,a01aa04,fluoruro de estaño,NaN,NaN,en,en
4,a,tracto alimentario y metabolismo,a01,preparaciones estomatológicas,a01a,preparaciones estomatológicas,a01aa,agentes profilácticos de caries,a01aa30,combinaciones,NaN,NaN,en,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,b,sangre y órganos formadores de sangre,b02,antihemorrágicos,b02b,vitamina k y otros hemostáticos,b02ba,vitamina K,b02ba01,fitomenadiona,20,mg,O,en
93,b,sangre y órganos formadores de sangre,b02,antihemorrágicos,b02b,vitamina k y otros hemostáticos,b02ba,vitamina K,b02ba01,fitomenadiona,20,mg,pag,en
94,b,sangre y órganos formadores de sangre,b02,antihemorrágicos,b02b,vitamina k y otros hemostáticos,b02ba,vitamina K,b02ba02,menadiona,10,mg,O,en
95,b,sangre y órganos formadores de sangre,b02,antihemorrágicos,b02b,vitamina k y otros hemostáticos,b02ba,vitamina K,b02ba02,menadiona,2,mg,pag,en


In [95]:
#Salvar ambas tablas en excel en pestañas diferentes
with pd.ExcelWriter('atc_clasificador.xlsx') as writer:
    df.to_excel(writer, sheet_name='atc5',index=False)
    df_esp.to_excel(writer, sheet_name='atc5_esp',index=False)